# Optimization results visualization

In [ ]:
# TODO :
# - Couleurs graphes
# - Date en axe x
# - Trouver la liste des chemins fichiers excels des résultats

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
import datetime

### Loading the results data

In [ ]:
Tmax = 168 #optimization for 1 week (7*24=168 hours)
Tmaxmax = Tmax + 24


def load_assets():
    return pd.read_excel('./sc_ref/inputs.xlsx', sheet_name="ASSETS").set_index('Name')

def load_timeseries():
    return pd.read_excel('./sc_ref/inputs.xlsx', sheet_name="TIMESERIES")

def read_results():
    list_p_out = []
    list_p_in = []
    list_stock_e = []
    list_assets_state = []

    excel_files = [f'./sc_ref/results_semaine_{n}.xlsx' for n in range(52)]#, './results_semaine_1.xlsx']
    for week, excel_file in enumerate(excel_files) :
        for (sheet, list_df) in [("P_OUT", list_p_out), ("P_IN", list_p_in), ("STOCK_E", list_stock_e)]:
            df = pd.read_excel(excel_file, sheet_name=sheet)
            df["Time"] = df["Time"] + Tmax*week
            df = df.set_index("Time")
            list_df.append(df)
        list_assets_state.append(pd.read_excel(excel_file, sheet_name="ASSETS_STATE").set_index('asset_name'))
    
    # Fusionner les résultats des différentes semaines et enlever les valeurs des anneaux de garde
    df_p_out = pd.concat(list_p_out).loc[lambda d: ~d.index.duplicated(keep="last")]
    df_p_in = pd.concat(list_p_in).loc[lambda d: ~d.index.duplicated(keep="last")]
    df_stock_e = pd.concat(list_stock_e).loc[lambda d: ~d.index.duplicated(keep="last")]
    df_assets_state = pd.concat(list_assets_state)

    return df_p_out, df_p_in, df_stock_e, df_assets_state

def x_absciss(df):
    start=datetime.datetime(2026,7,4,0,0,0)
    return pd.date_range(start, start + datetime.timedelta(hours=len(df.index)), freq=datetime.timedelta(hours=1))

In [ ]:
df_p_out, df_p_in, df_stock_e, df_assets_state = read_results()

In [ ]:
df_assets = load_assets()
df_assets.head()

In [ ]:
df_timeseries = load_timeseries()
df_timeseries.index = df_timeseries.index + 1
df_timeseries = df_timeseries.loc[:len(df_p_out)]

### Building graphs

In [ ]:
import plotly.express as px

# fig = px.colors.diverging.swatches()
# fig.show()

colors = {
    'interconnexion' : px.colors.diverging.RdGy[8],
    'PV' : px.colors.diverging.Portland[2],
    'Hydro' : px.colors.diverging.balance[2],
    'Wind' : px.colors.diverging.RdYlGn[8],
    'Waste' : px.colors.diverging.Earth[0],
    'Bio' : px.colors.diverging.delta[9],
    'tank' : px.colors.diverging.Spectral[10],
    'STEP' : px.colors.diverging.Spectral[10],
    'LOAD' : px.colors.diverging.RdYlGn[0],
    'CCG' : px.colors.diverging.Spectral[3],
    'CHP' : px.colors.diverging.Portland[3],
    'TAC' : px.colors.diverging.Spectral[3],
    'Pyro' : px.colors.diverging.Spectral[3],
    'Electrolizer' : px.colors.diverging.Spectral[3],
    'Methan' : px.colors.diverging.Spectral[2],
    'Vapo' : px.colors.diverging.Spectral[2],
    'Coal' : px.colors.diverging.RdGy[9],
    'Fioul' : px.colors.diverging.RdGy[8],
    'Nuclear' : px.colors.diverging.delta[6],
    'LNG' : px.colors.diverging.Tropic[6],
    'default' : px.colors.diverging.Picnic[8]
}

def get_color(description:str):
    for t in colors.keys():
        if t in description:
            return colors[t]
    return colors['default']

In [ ]:
for energy_vector, region in [('gas', 'n'), ('gas', 's'), ('h2', 'n'), ('h2', 's'), ('elec', None)]:
    fig = go.Figure()

    # Fatal load/production
    region_filter = {'n': 'NORTH', 's': 'SOUTH', None: ''}
    for fatal in [col for col in df_timeseries.columns if (((energy_vector).upper() in col) and (region_filter[region] in col))]:        
        if 'LOAD' in fatal :
            fig.add_trace(go.Bar(
                    x=x_absciss(df_timeseries),
                    y=-df_timeseries[fatal],
                    # base=0,
                    marker_color=get_color(fatal),
                    name=f"{fatal}"
                ))
        else :
            fig.add_trace(go.Bar(
                    x=x_absciss(df_timeseries),
                    y=df_timeseries[fatal],
                    # base=0,
                    marker_color=get_color(fatal),
                    name=f"P_out {fatal}"
                ))

    # Consuming assets
    for asset in df_assets[(df_assets['energy_in']==energy_vector) & (region is None or df_assets['region']==region)].index:
        fig.add_trace(go.Bar(
                x=x_absciss(df_p_in),
                y=-df_p_in[asset],
                # base=0,
                marker_color=get_color(asset),
                name=f"P_in {asset}"
            ))

    # Producing assets
    subset = df_assets[
        (df_assets['energy_out'] == energy_vector)
        & (region is None or df_assets['region'] == region)
    ].copy()

    subset['order'] = subset['family'].map({'inter': 0, 'disp': 1}).fillna(2)
    subset['asset_name'] = subset.index

    producing_assets = (
        subset
        .sort_values(
            by=['order', 'asset_name'],
            ascending=[True, False]   # ordre normal sur family, inverse alphabétique sur nom
        )
        .index
        .tolist()
    )
    
    for asset in producing_assets:
        fig.add_trace(go.Bar(
                x=x_absciss(df_p_out),
                y=df_p_out[asset],
                # base=0,
                marker_color=get_color(asset),
                name=f"P_out {asset}"
            ))
        
    # Storage assets
    for storage in df_assets[(df_assets['energy_out']==energy_vector) & (df_assets['family']=='stock') & (region is None or df_assets['region']==region)].index:
        fig.add_trace(go.Scatter(
                x=x_absciss(df_p_out),
                y=100 * df_stock_e[storage]/df_assets['emax'][storage],
                name=f"SOC {storage}",
                marker_color=get_color(asset),
                mode='lines', yaxis="y2"
            ))
        
    # Imports/exports
    other_region = {'n':'s', 's':'n'}
    if region :
        port = f"{energy_vector.capitalize()}_interconnexion_{other_region[region].upper()}"
        fig.add_trace(go.Bar(
                x=x_absciss(df_p_in),
                y=df_p_in[port],
                # base=0,
                marker_color=get_color(port),
                name=f"Import {port}"
            ))
        fig.add_trace(go.Bar(
                x=x_absciss(df_p_in),
                y=-df_p_out[port],
                # base=0,
                marker_color=get_color(port),
                name=f"Export {port}"
            ))


    fig.update_layout(
        title=f"{energy_vector.capitalize()} management in {region_filter[region].capitalize()} Zootopia (producer convention)",
        xaxis_title="Time",
        yaxis=dict(
            title="Power [MW]",
            side="left",
            ticksuffix=" MW"
        ),
        yaxis2=dict(
            title="State Of Charge [%]",
            overlaying="y",
            side="right",
            showgrid=False,
            range=[0, 100],
            ticksuffix=" %"
        ),
        barmode='relative',
        template = 'plotly_dark',
        bargap = 0,
    )
    fig.update_traces(marker_line_width=0, selector=dict(type="bar"))

    fig.show()